# 0. Import thư viện

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

# 1. Data Preprocessing
## 1.0. Data Loading

In [3]:
df = pd.read_csv('data/kill_match_stats.csv')

In [ ]:
df.head()

In [ ]:
df.drop(columns = ['killer_name', 'victim_name', 'killer_placement', 'victim_placement', 'match_id'], inplace = True)

In [ ]:
df = df[df['map'] == 'ERANGEL'].reset_index(drop = True)

In [ ]:
df.drop(columns = ['map'], inplace = True)

In [ ]:
df.rename(columns = {
    'killer_position_x': 'kx', 
    'killer_position_y': 'ky',
    'victim_position_x': 'vx',
    'victim_position_y': 'vy'
},inplace = True)

In [9]:
df.head()

,killed_by,kx,ky,time,vx,vy
0,Down and Out,496989.8,312569.7,1035,497385.4,331528.2
1,M16A4,496989.8,312569.7,1035,497819.4,331981.3
2,AKM,460416.7,414748.8,1422,459817.9,414426.3
3,AKM,488034.1,347220.3,1210,487444.2,347651.0
4,SKS,501062.9,425078.6,1818,493043.4,434458.1


<span style="color:red">Câu lệnh bên dưới chỉ thực hiện một lần duy nhất trên máy một thành viên duy nhất. Sau khi thực hiện, tiến hành lưu trữ df thành file kill_match_stats_v1.csv, các thành viên khác chỉ cần load file này vào là được, không cần chạy lại câu lệnh `to_csv` để tránh mất thời gian<span>

In [10]:
df.to_csv('data/kill_match_stats_v1.csv', index = False)

Test Markdown

# 0. Import thư viện

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

## 1.1. Data Cleaning

In [ ]:
df = pd.read_csv('data/kill_match_stats_v1.csv')
df.head()

### 1.1.1. Missing values

Kiểm tra missing value trên tất cả các cột

In [ ]:
df.isnull().sum()

In [ ]:
print(f'Số dòng của df: {df.shape[0]:,}')

Ở đây có khoảng hơn 700000 missing value ở 2 cột `kx` và `ky` so với khoảng 10,000,000 dòng dữ liệu chỉ chiếm 7%, ta có thể dễ dàng loại bỏ các dòng này. Tuy nhiên ta có thể quan sát các dòng này một chút

In [ ]:
df[df['kx'].isnull()]['killed_by'].value_counts()

Quan sát cột `killed_by` thấy nguyên nhân gây kill ở các dòng `kx` và `ky` trống ta thấy chủ yếu các nguyên nhân này thuộc vào loại Zone (các vùng gây sát thương), các loại xe, các loại vũ khí ném và các nguyên nhân tự thân (`'Falling': Ngã, Drown: Đuối nước`). Đối vơi các nguyên nhân này, không có killer, do đó không có tọa độ của killer. Ta có thể tạm gán tọa độ `kx` và `ky` của các dòng này bằng với tọa độ của victim `vx` và `vy`

In [ ]:
df.loc[df['kx'].isnull(), 'kx'] = df['vx']
df.loc[df['ky'].isnull(), 'ky'] = df['vy']

### 1.1.2. Duplicated values

In [ ]:
df.duplicated().sum()

Dùng hàm `duplicated` để kiểm tra các dòng trùng nhau, ta thấy có 257932 dòng dữ liệu bị trùng. Tuy nhiên ta có thể quan sát kĩ hơn các dòng này 

In [ ]:
data = df[df.duplicated()]

In [ ]:
data[['kx', 'ky', 'vx', 'vy']].value_counts()

Quan sát các giá trị khác nhau của các cột tọa độ, ta nhận thấy đa số các cột trùng nhau có tọa độ killer và victim nằm ở (0, 0), nghĩa là ở góc bản đồ, nơi không có địa hình (sẽ được minh họa kỹ hơn ở phần trực quan hóa dữ liệu). Đây có thể là lỗi trong quá trình game vận hành, số lượng các dòng bị lỗi cũng chỉ khoảng 2,5% dữ liệu, ta có thể đơn giản xóa đi các dòng này

In [ ]:
df.drop_duplicates(inplace = True, ignore_index = True)

### 1.1.3. Outliers

Hiện tại `df` có 1 cột categorical là `killed_by` và 5 cột còn lại là numerical. Đối với cột `categorical`, nhóm sử dụng hàm value_counts để xác định outliers, còn đối với các cột numerical, nhóm sử dụng boxplot, kết hợp với thông tin về bản đồ của game để xác định và xử lí các outliers

In [ ]:
print(df['killed_by'].value_counts().sort_values())

Quan sát cột `killed_by`, ta thấy một vài outlier là `death.PlayerMale_A_C`, `death.RedZoneBomb_C`, `Aquarial`, `death.ProjMolotov`, `Boat`, `death.Buf_FireDOT_C`. Tuy là outliers nhưng các nguyên nhân gây kill này vẫn tồn tại trong game, do ít được sử dụng nên được ghi nhận ít. Nhóm có tra cứu thì nhận thấy ngoài `death.PlayerMale_A_C` và `death.Buff_FireDOT_C`, các nguyên nhân còn lại đều hợp lệ, chỉ bị sai tên, sẽ được sửa ở phần Data Quality bên dưới. Do dó ở cột `killed_by`, nhóm chỉ thực hiện xóa các dòng có `death.PlayerMale_A_C` và `death.Buff_FireDOT_C` của cột `killed_by`

In [ ]:
df = df[(df['killed_by'] != 'death.PlayerMale_A_C') & (df['killed_by'] != 'death.Buff_FireDOT_C') ]

Đối với 5 cột numerical thì chỉ có cột `time` là khác ý nghĩa với 4 cột tọa độ còn lại, ta gom nhóm 4 cột tọa độ vào cùng một boxplot

In [ ]:
sns.boxplot(x = 'variable', y = 'value', data = pd.melt(df[['kx', 'ky', 'vx', 'vy']]));

Quan sát các giá trị tọa độ, ta nhận ra đa số các giá trị đều nằm trong khoảng từ 0 tới 800,000. Đây cũng là khoảng dữ liệu hợp lệ đã được đề cập trong mô tả dữ liệu của tác giả. Các giá trị ngoại lai lớn hơn 0 và nhỏ hơn 800,000 cũng xuất hiện không nhiều, không gây ảnh hưởng tới việc phân tích dữ liệu về sau, nên ta có thể chấp nhận các giá trị này. Do đó ở đây nhóm chỉ loại bỏ các giá trị tọa độ lớn hơn 800,000 hoặc nhỏ hơn 0.

In [ ]:
df = df[(df['kx'] <= 800000) & (df['kx'] >= 0) 
        & (df['ky'] <= 800000) & (df['ky'] >= 0)
        & (df['vx'] <= 800000) & (df['vx'] >= 0)
        & (df['vy'] <= 800000) & (df['vy'] >= 0)]

In [ ]:
sns.boxplot(x = df['time']);

Cột `time` không có outlier do đó ta không cần phải xử lí